In [ ]:
import numpy as np
import pandas as pd

#data visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

#to avoid warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import pandas as pd
# train_df = pd.read_csv("data/raw/train.csv")
# test_df = pd.read_csv("data/raw/test.csv")


In [ ]:
# column_labels = train_df.columns.tolist()[2:]
# train_df[column_labels].sum().sort_values()

In [ ]:
# print(train_df['toxic'].value_counts())

In [ ]:
# # Split toxic and clean
# train_toxic = train_df[train_df['toxic'] == 1]
# train_non_toxic = train_df[train_df['toxic'] == 0]

# #  Undersample clean class to match toxic count
# train_non_toxic_sampled = train_non_toxic.sample(n=len(train_toxic), random_state=42)

# # : Combine both and shuffle
# train_balanced = pd.concat([train_toxic, train_non_toxic_sampled], axis=0)
# train_balanced = train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# #  Confirm balance
# print(train_balanced['toxic'].value_counts())


In [ ]:
import pandas as pd

# Load the CSV from Colab's file system
train_balanced = pd.read_csv("/content/train_balanced.csv")

# Preview the data
train_balanced.head()


In [ ]:
print(train_balanced.columns)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    train_balanced['comment_text'],  # or dataframe['comment_text'] if it's the same
    train_balanced.iloc[:, 2:],      # all 6 label columns
    test_size=0.25,
    random_state=42
)



In [ ]:
# Split test set into 50% validation and 50% final test
val_texts, final_test_texts, val_labels, final_test_labels = train_test_split(
    test_texts,
    test_labels,
    test_size=0.5,
    random_state=42
)


Tokenization and Encoding

In [ ]:
def tokenize_and_encode(tokenizer, comments, labels, max_length=128):
    # Tokenize the list of comments at once (batch encoding is more efficient)
    encoding = tokenizer(
        comments.tolist(),                      # Ensure it's a list of strings
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',                   # Use padding parameter instead of pad_to_max_length
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Convert labels to torch tensors
    labels = torch.tensor(labels.values, dtype=torch.float32)

    return encoding['input_ids'], encoding['attention_mask'], labels


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Initialize Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#  Initialize Model (6 labels for multi-label classification)
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=6,
    problem_type="multi_label_classification"  # Important for multi-label setup
)

# Device Setup (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#  Tokenize and Encode Each Dataset
train_input_ids, train_attention_masks, train_labels_tensor = tokenize_and_encode(
    tokenizer, train_texts, train_labels)

val_input_ids, val_attention_masks, val_labels_tensor = tokenize_and_encode(
    tokenizer, val_texts, val_labels)

test_input_ids, test_attention_masks, test_labels_tensor = tokenize_and_encode(
    tokenizer, test_texts, test_labels)

# Check Shapes
print('Training Comments :', train_texts.shape)
print('Input Ids         :', train_input_ids.shape)
print('Attention Mask    :', train_attention_masks.shape)
print('Labels            :', train_labels_tensor.shape)


Creating Pytorch Data Loaders

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Set batch size
batch_size = 32

# Train DataLoader
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Validation DataLoader
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Test DataLoader
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# View one batch of training data
print('Batch Size :', train_loader.batch_size)

batch = next(iter(train_loader))

print('Each Input ids shape      :', batch[0].shape)  # (batch_size, sequence_length)
print('Input ids [0]             :\n', batch[0][0])
print('Decoded text [0]          :\n', tokenizer.decode(batch[0][0], skip_special_tokens=True))
print('Corresponding Attention Mask:\n', batch[1][0])
print('Corresponding Label [0]   :', batch[2][0])


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


Model Training

In [ ]:
# Function to Train the Model
def train_model(model, train_loader, optimizer, device, num_epochs):
    # Loop through the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()
        # Initialize total loss for the current epoch
        total_loss = 0

        # Loop through the batches in the training data
        for batch in train_loader:
            input_ids, attention_mask, labels = [t.to(device) for t in batch]

            optimizer.zero_grad()

            outputs = model(
                input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()  # Set the model to evaluation mode
        val_loss = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [
                    t.to(device) for t in batch]

                outputs = model(
                    input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()
        # Print the average loss for the current epoch
        print(
            f'Epoch {epoch+1}, Training Loss: {total_loss/len(train_loader)},Validation loss:{val_loss/len(val_loader)}')


# Call the function to train the model
train_model(model, train_loader, optimizer, device, num_epochs=3)

Model Evaluation

In [ ]:
# Evaluate the Model
def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode

    true_labels = []
    predicted_probs = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [t.to(device) for t in batch]

            # Get model's predictions
            outputs = model(input_ids, attention_mask=attention_mask)
            # Use sigmoid for multilabel classification
            predicted_probs_batch = torch.sigmoid(outputs.logits)
            predicted_probs.append(predicted_probs_batch.cpu().numpy())

            true_labels_batch = labels.cpu().numpy()
            true_labels.append(true_labels_batch)

    # Combine predictions and labels for evaluation
    true_labels = np.concatenate(true_labels, axis=0)
    predicted_probs = np.concatenate(predicted_probs, axis=0)
    predicted_labels = (predicted_probs > 0.5).astype(
        int)  # Apply threshold for binary classification

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')

    # Print the evaluation metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')


# Call the function to evaluate the model on the test data
evaluate_model(model, test_loader, device)

In [ ]:
# Save the tokenizer and model in the same directory
output_dir = "Saved_model"
# Save model's state dictionary and configuration
model.save_pretrained(output_dir)
# Save tokenizer's configuration and vocabulary
tokenizer.save_pretrained(output_dir)

In [ ]:
# Load the tokenizer and model from the saved directory
model_name = "Saved_model"
Bert_Tokenizer = BertTokenizer.from_pretrained(model_name)
Bert_Model = BertForSequenceClassification.from_pretrained(
    model_name).to(device)

In [ ]:
def predict_user_input(input_text, model=Bert_Model, tokenizer=Bert_Tokenizer, device=device):
    user_input = [input_text]

    user_encodings = tokenizer(
        user_input, truncation=True, padding=True, return_tensors="pt")

    user_dataset = TensorDataset(
        user_encodings['input_ids'], user_encodings['attention_mask'])

    user_loader = DataLoader(user_dataset, batch_size=1, shuffle=False)

    model.eval()
    with torch.no_grad():
        for batch in user_loader:
            input_ids, attention_mask = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)

    predicted_labels = (predictions.cpu().numpy() > 0.5).astype(int)
    labels_list = ['toxic', 'severe_toxic', 'obscene',
                   'threat', 'insult', 'identity_hate']
    result = dict(zip(labels_list, predicted_labels[0]))
    return result


text = 'Are you insane!'
predict_user_input(input_text=text)

In [ ]:
import shutil

# Zip the folder
shutil.make_archive('saved_model', 'zip', 'Saved_model')


In [ ]:
from google.colab import files

# Download to your local machine
files.download('saved_model.zip')


In [ ]:
predict_user_input(input_text='How are you?')

In [ ]:
text = "Such an Idiot person"
predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text,
                   device=device)